# glm-met

Example use of the glm-met Python package to download meteorological data and format it for use with the General Lake Model (GLM).

In [1]:
# install glm-met - comment if this step is not required
# !pip install glm-met
!pip install -i https://test.pypi.org/simple/ glm-met==0.0.4

Looking in indexes: https://test.pypi.org/simple/
  Attempting uninstall: glm-met
    Found existing installation: glm-met 0.0.3
    Uninstalling glm-met-0.0.3:
      Successfully uninstalled glm-met-0.0.3

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## Historical data

Here, we'll be working with the open-meteo Historical API which lets you download historical meteorological data based on the ERA-Land reanalysis product.

First, we need to create a `glm_met.openmeteo.historical.Historical` object, which stores API query attributes and has methods for downloading and processing the data.

In [2]:
import pprint
import pandas as pd
import os
import glm_met.openmeteo.historical as historical
import glm_met.openmeteo.climate_change as climate

variables = [
    "temperature_2m",
    "relativehumidity_2m",
    "precipitation",
    "cloudcover",
    "et0_fao_evapotranspiration",
    "windspeed_10m",
    "winddirection_10m",
    "windgusts_10m",
    "shortwave_radiation",
    "direct_normal_irradiance"
]

tmp_historical = historical.Historical(
        location=(116.69115540712589, -34.22581232495377),
        date_range=("2020-01-01", "2020-01-31"),
        variables=variables,
        met_data=None,
        glm_met_data=None,
        hourly=True
    )

Once we have created the `Historical` object, `tmp_historical`, we can call its `get_variables()` method to make a request to the open-meteo Historical API and download the data. The downloaded data will be stored in `tmp_historical`'s `met_data` attribute. The `met_data` attribute's type is of `MetData` which has two attributes. `tmp_historical.met_data.metadata` is a dict of metadata about the request to the open-meteo Historical API and `tmp_historical.met_data.data` is a DataFrame object of meteorological data.

In [3]:
tmp_historical.get_variables(request_settings=None)

In [4]:
# view the metadata
pprint.pprint(tmp_historical.met_data.metadata)

{'elevation': 208.0,
 'generationtime_ms': 1.7659664154052734,
 'hourly_units': {'cloudcover': '%',
                  'direct_normal_irradiance': 'W/m²',
                  'et0_fao_evapotranspiration': 'mm',
                  'precipitation': 'mm',
                  'relativehumidity_2m': '%',
                  'shortwave_radiation': 'W/m²',
                  'temperature_2m': '°C',
                  'time': 'iso8601',
                  'winddirection_10m': '°',
                  'windgusts_10m': 'm/s',
                  'windspeed_10m': 'm/s'},
 'latitude': -34.2,
 'longitude': 116.70001,
 'timezone': 'Australia/Perth',
 'timezone_abbreviation': 'AWST',
 'utc_offset_seconds': 28800}


In [5]:
# view the data
tmp_historical.met_data.data.head()

,time,temperature_2m,relativehumidity_2m,precipitation,cloudcover,et0_fao_evapotranspiration,windspeed_10m,winddirection_10m,windgusts_10m,shortwave_radiation,direct_normal_irradiance
0,2020-01-01T00:00,15.4,80,0.0,5,0.0,1.33,167,3.7,0.0,0.0
1,2020-01-01T01:00,14.6,85,0.0,0,0.0,1.30,113,2.6,0.0,0.0
2,2020-01-01T02:00,14.1,88,0.0,0,0.0,1.08,112,2.0,0.0,0.0
3,2020-01-01T03:00,14.2,89,0.0,2,0.0,0.85,111,1.9,0.0,0.0
4,2020-01-01T04:00,14.1,89,0.0,38,0.0,0.73,106,1.9,0.0,0.0


The GLM model requires meteorological data with a specific format and units. If the required meteorological variables to run GLM have been requested from open-meteo, `tmp_historical`'s `convert_to_glm()` method can be used reformat the data. The GLM formatted data is a DataFrame that is stored in `tmp_historical`'s `glm_met_data` attribute.

In [6]:
# convert data to GLM format
tmp_historical.convert_to_glm()

# visualise GLM formatted data
tmp_historical.glm_met_data.head()

,time,ShortWave,Cloud,AirTemp,RelHum,WindSpeed,Rain
0,2020-01-01 00:00:00,0.0,0.05,15.4,80,1.33,0.0
1,2020-01-01 01:00:00,0.0,0.00,14.6,85,1.30,0.0
2,2020-01-01 02:00:00,0.0,0.00,14.1,88,1.08,0.0
3,2020-01-01 03:00:00,0.0,0.02,14.2,89,0.85,0.0
4,2020-01-01 04:00:00,0.0,0.38,14.1,89,0.73,0.0


Finally, we can save the GLM formatted meteorological data to a csv file for use with the GLM model.

In [7]:
tmp_historical.write_glm_met(path=os.getcwd(), zip_f=False, fname="met.csv")

if "met.csv" in os.listdir(os.getcwd()):
    print("met.csv downloaded OK")

met.csv downloaded OK


## Climate Change data

First, instantiate a `ClimateChange` object. Here, we'll use the default variables. However, if you want to use a custom selection check the <a href="https://open-meteo.com/en/docs/climate-api#" target="_blank">open-meteo Climate API docs</a> for a list of valid meteorological variables names.

In [8]:
clim = climate.ClimateChange(
    location=(116.691155, -34.225812),
    date_range=("2022-01-01", "2032-12-31"),
    met_data=None,
    glm_met_data=None
)

In [9]:
clim.get_variables(request_settings=None)
clim.convert_to_glm()

In [10]:
# raw climate change data
clim.met_data.data.head()

,time,shortwave_radiation_sum_CMCC_CM2_VHR4,cloudcover_mean_CMCC_CM2_VHR4,temperature_2m_mean_CMCC_CM2_VHR4,temperature_2m_max_CMCC_CM2_VHR4,temperature_2m_min_CMCC_CM2_VHR4,relative_humidity_2m_mean_CMCC_CM2_VHR4,windspeed_10m_mean_CMCC_CM2_VHR4,precipitation_sum_CMCC_CM2_VHR4,et0_fao_evapotranspiration_sum_CMCC_CM2_VHR4,...,et0_fao_evapotranspiration_sum_MPI_ESM1_2_XR,shortwave_radiation_sum_NICAM16_8S,cloudcover_mean_NICAM16_8S,temperature_2m_mean_NICAM16_8S,temperature_2m_max_NICAM16_8S,temperature_2m_min_NICAM16_8S,relative_humidity_2m_mean_NICAM16_8S,windspeed_10m_mean_NICAM16_8S,precipitation_sum_NICAM16_8S,et0_fao_evapotranspiration_sum_NICAM16_8S
0,2022-01-01,None,None,23.6,30.6,17.6,46,7.43,1.66,5.46,...,6.91,28.45,18,20.3,26.7,13.6,63,4.43,0.0,6.05
1,2022-01-02,None,None,21.4,27.0,17.1,75,3.80,0.63,5.24,...,8.01,29.38,9,15.6,21.8,9.5,56,4.11,0.0,5.57
2,2022-01-03,None,None,20.8,27.8,14.7,74,3.38,0.23,6.15,...,7.38,29.54,19,20.3,32.3,12.8,51,2.58,0.0,7.13
3,2022-01-04,None,None,19.4,27.8,13.4,71,2.79,0.68,6.50,...,6.39,29.37,9,19.7,29.1,13.3,60,4.03,0.0,6.37
4,2022-01-05,None,None,18.1,24.3,13.9,74,2.96,0.57,5.21,...,6.09,29.20,12,20.9,29.1,14.1,48,4.60,0.0,6.32


The `climate.glm_met_data` attribute stores the results of calling the `convert_to_glm()` method. It is a dict with each key representing a climate model and the value a `pd.DataFrame` of meteorological varaibles for the given model.

In [11]:

clim.glm_met_data.keys()

dict_keys(['CMCC_CM2_VHR4', 'FGOALS_f3_H', 'HiRAM_SIT_HR', 'MRI_AGCM3_2_S', 'EC_Earth3P_HR', 'MPI_ESM1_2_XR', 'NICAM16_8S'])

In [12]:
clim.glm_met_data["CMCC_CM2_VHR4"].head()

,time,ShortWave,Cloud,AirTemp,RelHum,WindSpeed,Rain
0,2022-01-01,None,NaN,23.6,46,7.43,0.00166
1,2022-01-02,None,NaN,21.4,75,3.80,0.00063
2,2022-01-03,None,NaN,20.8,74,3.38,0.00023
3,2022-01-04,None,NaN,19.4,71,2.79,0.00068
4,2022-01-05,None,NaN,18.1,74,2.96,0.00057
